#### `Return the number of distinct words`

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [2]:
spark = SparkSession.builder.appName(
    "Counting word occurences from a book."
).getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [22]:
# If you need to read multiple text files, replace `1342-0` by `*`.
results = (
    spark.read.text("./DataAnalysisPythonPySpark/data/data/gutenberg_books/1342-0.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
    .where(F.col("word") != "")
    # .groupby(F.col("word"))
    .distinct()
    .count()
)
results
# results.orderBy("count", ascending=False).show(10)
# results.coalesce(1).write.csv("./results_single_partition.csv")

6595

##### ```Getting the words that appears only once.```

In [36]:
# If you need to read multiple text files, replace `1342-0` by `*`.
results = (
    spark.read.text("./DataAnalysisPythonPySpark/data/data/gutenberg_books/1342-0.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
    .where(F.col("word") != "")
    .groupby(F.col("word"))
    # .distinct()
    .count()
    # .where(F.col('count') == 1)
    # .orderBy('word', ascending=True)
)
print(results.show(10))
print(results.count())


+---------+-----+
|     word|count|
+---------+-----+
|   online|    4|
|     some|  203|
|    still|   72|
|      few|   72|
|     hope|  122|
|    those|   60|
| cautious|    4|
|   lady's|    8|
|imitation|    1|
|      art|    3|
+---------+-----+
only showing top 10 rows

None
6595


In [40]:
results.withColumn('first_letter', F.substring(F.col('word'), 1, 1)).groupby(F.col('first_letter')).sum() \
    .orderBy('Sum(Count)', ascending=False).show(5)


+------------+----------+
|first_letter|sum(count)|
+------------+----------+
|           t|     16101|
|           a|     13684|
|           h|     10419|
|           w|      9091|
|           s|      8791|
+------------+----------+
only showing top 5 rows



In [41]:
results.withColumn('First_letter_Vowel',
F.substring(F.col('word'), 1, 1).isin(['a','e','i','o','u']),) \
    .groupby(F.col("First_letter_Vowel")).sum().show()

+------------------+----------+
|First_letter_Vowel|sum(count)|
+------------------+----------+
|              true|     33522|
|             false|     88653|
+------------------+----------+



In [48]:
logs = spark.read.csv('./DataAnalysisPythonPySpark/data/data/broadcast_logs/BroadcastLogs_2018_Q3_M8_sample.csv', sep='|',
header=True, inferSchema=True, timestampFormat='yyyy-MM-dd')

In [65]:
logs.show(5, vertical=True)

-RECORD 0--------------------------------------
 BroadcastLogID         | 1196192316           
 LogServiceID           | 3157                 
 LogDate                | 2018-08-01 00:00:00  
 SequenceNO             | 1                    
 AudienceTargetAgeID    | 4                    
 AudienceTargetEthnicID | null                 
 CategoryID             | 13                   
 ClosedCaptionID        | 3                    
 CountryOfOriginID      | 3                    
 DubDramaCreditID       | null                 
 EthnicProgramID        | null                 
 ProductionSourceID     | 10                   
 ProgramClassID         | 19                   
 FilmClassificationID   | null                 
 ExhibitionID           | 2                    
 Duration               | 02:00:00.0000000     
 EndTime                | 08:00:00.0000000     
 LogEntryDate           | 2018-08-01 00:00:00  
 ProductionNO           | A39082               
 ProgramTitle           | Newlywed and D

In [61]:
sample = spark.read.csv('./sample.csv',header=True,inferSchema=True,
quote='$')

In [62]:
print(sample.printSchema())
print(sample.show())

root
 |-- Item: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)

None
+---------------+--------+-----+
|           Item|Quantity|Price|
+---------------+--------+-----+
|Banana, organic|       1| 0.99|
|           Pear|       7| 1.24|
|Cake, chocolate|       1| 14.5|
+---------------+--------+-----+

None


In [64]:
print(logs.select('BroadcastLogID', 'LogServiceID', 'LogDate').show(5,False))

+--------------+------------+-------------------+
|BroadcastLogID|LogServiceID|            LogDate|
+--------------+------------+-------------------+
|    1196192316|        3157|2018-08-01 00:00:00|
|    1196192317|        3157|2018-08-01 00:00:00|
|    1196192318|        3157|2018-08-01 00:00:00|
|    1196192319|        3157|2018-08-01 00:00:00|
|    1196192320|        3157|2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 5 rows

None
